In [ ]:
import pandas as pd
import numpy as np
from plotly import __version__
import cufflinks as cf
from plotly.offline import download_plotlyjs,init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
#drop non-shopper IOs
def drop_nonshopper(df):
    df['Insertion Order']=df['Insertion Order'].str.lower()  
    return df[df['Insertion Order'].str.contains("shopper")]

In [ ]:
#define extract name plate function

def extract_name_plate(io):
    if "ranger" in io:
        return "Ranger"
    elif "everest" in io:
        return "Everest"
    elif "focus" in io:
        return "Focus"
    elif "escort" in io:
        return "Escort"
    elif "ecosport" in io:
        return "Ecosport"
    elif "transit" in io:
        return "Transit"
    elif "mondeo" in io:
        return "Mondeo"
    else:
        return "Others"

In [ ]:
pc_score = {}
with open("pc.txt") as f:
    for line in f:
        l=line.split()
        val=l[len(l)-1]
        key=line.replace(" "+val,"")
        key=key.rstrip()
        pc_score[(key)]=val
        
mobile_score = {}
with open("mobile.txt") as f:
    for line in f:
        l=line.split()
        val=l[len(l)-1]
        key=line.replace(" "+val,"")
        key=key.rstrip()
        mobile_score[(key)] = val

In [ ]:
#define create tactic column

def add_tactic_column(df):
    df['Tactic']=df.apply(lambda row: 'RTG LAL Intender' if (('Intender' in str(row['Line Item'])) and ('Custom HIP' in str(row['Line Item'])))
                          else ('RTG LAL Ready to Buy' if (('Ready' in str(row['Line Item'])) and ('Custom HIP' in str(row['Line Item'])))
                              else ('RTG LAL Researcher' if (('Research' in str(row['Line Item'])) and ('Custom HIP' in str(row['Line Item'])))
                                else ('RTG LAL Dealership' if (('Dearlership' in str(row['Line Item'])) and ('Custom HIP' in str(row['Line Item'])))
                                    else ('RTG '+ 'Ready to Buy' if (('RTG' in str(row['Line Item'])) and ('Ready' in str(row['Line Item'])))
                                        else ('RTG '+ 'Researcher' if (('RTG' in str(row['Line Item'])) and ('Research' in str(row['Line Item'])))
                                            else ('RTG '+ 'Dealership' if (('RTG' in str(row['Line Item'])) and ('Dealership' in str(row['Line Item'])))
                                                else ('RTG '+ 'Intender' if (('RTG' in str(row['Line Item'])) and ('Intender' in str(row['Line Item'])))
                                                    else ('RTG '+ 'Others' if (('RTG' in str(row['Line Item'])))
                                                        else ('Competitor '+ 'Keywords' if (('Keywords' in str(row['Line Item'])) or ('Keyword' in str(row['Line Item'])))
                                                            else ('Competitor '+ 'Categories' if (('Categories' in str(row['Line Item'])) or ('Category' in str(row['Line Item'])))
                                                                else ('Competitor '+ 'LAL' if ('LAL' in str(row['Line Item']))
                                                                        else ('Competitor '+ 'Custom Affinity' if (('Custom_Affinity' or 'Custom Affinity') in str(row['Line Item']))
                                                                            else ('Competitor '+ 'In market' if (('In market' in str(row['Line Item'])) or ('In Market' in str(row['Line Item'])) or ('InMarket' in str(row['Line Item']))or ('In-Market' in str(row['Line Item']))or ('In-market' in str(row['Line Item'])))
                                                                                else ('Competitor '+ 'PG Deals' if ('PG Deals' in str(row['Line Item']))
                                                                                    else ('Competitor '+ 'Custom Intent' if (('Custom Intent' in str(row['Line Item'])) or ('Custom intent' in str(row['Line Item'])))
                                                                                        else ('Competitor '+ 'Website' if (('Website' in str(row['Line Item'])) or ('website' in str(row['Line Item'])))
                                                                                            else ('Competitor '+ 'Affinity' if ('Affinity' in str(row['Line Item']))
                                                                                                else 'Competitor (Others)'
                                                                                                )
                                                                                            )
                                                                                        )
                                                                                    )
                                                                                )
                                                                            )
                                                                        )
                                                                    )
                                                                )
                                                            )
                                                        )
                                                    )
                                                )
                                            )
                                       )
                                   )
                               )
                            ,axis=1)
    return df


In [ ]:
#define create strategy column

def add_strategy_column(df):
    df['Strategy']=df.apply(lambda row: 'AAM LAL' if (('Custom HIP') or ('Custom Hip')) in str(row['Line Item'])
                          else ('Prospecting' if 'Competitor' in row['Line Item'] else 'AAM'
                              ),axis=1)
    return df

In [ ]:
#define create PI weight column

def add_weight_column(df):
    df['weight']=df.apply(lambda row: 0 if ((row['Floodlight Activity Name'] == 'Unknown') or (pc_score.get(str(row['Floodlight Activity Name'])) is None))
                          else (float(pc_score.get(str(row['Floodlight Activity Name']))) if row['Device Type']=='Desktop' else( float(mobile_score.get(str(row['Floodlight Activity Name']))) )
                              ),axis=1)
    return df

In [ ]:
#define create name plate column

def add_name_plate_column(df):
    df['Name Plate']=df.apply(lambda row: extract_name_plate(str(row['Insertion Order'])),axis=1)
    return df
    

In [ ]:
#define create browser family column

def add_browser_family_column(df):
    df['browser_family']=df.apply(lambda row: 'Safari' if 'Safari' in str(row['Browser'])
                          else ('Internet Explorer' if 'Internet Explorer' in row['Browser'] else row['Browser']
                              ),axis=1)
    return df

In [ ]:
#define create OS family column

def add_os_family_column(df):
    df['OS Family']=df.apply(lambda row: 'Android' if 'Android' in str(row['Operating System'])
                          else ('iOS' if 'iOS' in row['Operating System'] else ('Windows' if 'Windows' in str(row['Operating System']) else row['Operating System'])
                              ),axis=1)
    return df

In [ ]:
#define create total PI column

def add_pi_column(df):
    df['PI']=df.apply(lambda row: row['weight']*row['Total Conversions'],axis=1)
    return df

In [ ]:
#define create table for device
def pivot_device(df):
    pivot_device=df.pivot_table(index='Device Type',values=['PI','Media Cost'],aggfunc=np.sum)
    pivot_device['CPPI']=pivot_device.apply(lambda row: row['Media Cost'] / row['PI'] if row['PI']!=0 else 0,axis=1)
    pivot_device.sort_values(['Media Cost'],ascending=False)
    return pivot_device

In [ ]:
#define create table for strategy
def pivot_strategy(df):
    pivot_strategy=df.pivot_table(index='Strategy',values=['PI','Media Cost'],aggfunc=np.sum)
    pivot_strategy['CPPI']=pivot_strategy.apply(lambda row: row['Media Cost'] / row['PI'] if row['PI']!=0 else 0,axis=1)
    pivot_strategy=pivot_strategy.sort_values(['Media Cost'],ascending=False)
    return pivot_strategy

In [ ]:
#define create table for tactic
def pivot_tactic(df):
    pivot_tactic=df.pivot_table(index='Tactic',values=['PI','Media Cost'],aggfunc=np.sum)
    pivot_tactic['CPPI']=pivot_tactic.apply(lambda row: row['Media Cost'] / row['PI'] if row['PI']!=0 else 0,axis=1)
    pivot_tactic=pivot_tactic.sort_values(['Media Cost'],ascending=False)
    return pivot_tactic

In [ ]:
#define create table for browser family
def pivot_browser_family(df):
    pivot=df.pivot_table(index='browser_family',values=['PI','Media Cost'],aggfunc=np.sum)
    pivot['CPPI']=pivot.apply(lambda row: row['Media Cost'] / row['PI'] if row['PI']!=0 else 0,axis=1)
    pivot=pivot.sort_values(['Media Cost'],ascending=False)
    return pivot

In [ ]:
#define create table for OS family
def pivot_os_family(df):
    pivot=df.pivot_table(index='OS Family',values=['PI','Media Cost'],aggfunc=np.sum)
    pivot['CPPI']=pivot.apply(lambda row: row['Media Cost'] / row['PI'] if row['PI']!=0 else 0,axis=1)
    pivot=pivot.sort_values(['Media Cost'],ascending=False)
    return pivot

In [ ]:
#define create table for URL
def pivot_url(df):
    pivot=df.pivot_table(index='App/URL',values=['PI','Media Cost'],aggfunc=np.sum)
    pivot['CPPI']=pivot.apply(lambda row: row['Media Cost'] / row['PI'] if row['PI']!=0 else 0,axis=1)
    pivot=pivot.sort_values(['Media Cost'],ascending=False)
    return pivot

In [ ]:
#define create table for tactic-device
def pivot_tactic_device(df):
    pivot_tactic=df.pivot_table(index=['Tactic','Device Type'],values=['PI','Media Cost'],aggfunc=np.sum)
    pivot_tactic['CPPI']=pivot_tactic.apply(lambda row: row['Media Cost'] / row['PI'] if row['PI']!=0 else 0,axis=1)
    pivot_tactic=pivot_tactic.sort_values(['Tactic'],ascending=True)
    return pivot_tactic

In [ ]:
#define create table for strategy-device
def pivot_strategy_device(df):
    pivot_tactic=df.pivot_table(index=['Strategy','Device Type'],values=['PI','Media Cost'],aggfunc=np.sum)
    pivot_tactic['CPPI']=pivot_tactic.apply(lambda row: row['Media Cost'] / row['PI'] if row['PI']!=0 else 0,axis=1)
    pivot_tactic=pivot_tactic.sort_values(['Strategy'],ascending=True)
    return pivot_tactic

In [ ]:
#define create table for pixel
def pivot_pixel(df):
    df=df[df['Floodlight Activity Name']!='Unknown']
    pivot_tactic=df.pivot_table(index=['Floodlight Activity Name','Device Type','weight'],values=['Total Conversions','PI'],aggfunc=np.sum)
    pivot_tactic=pivot_tactic.sort_values(['Floodlight Activity Name'],ascending=True)
    pivot_tactic.rename(columns={"PI": "Total PI"})
    return pivot_tactic

In [ ]:
#define write to excel function

def write_to_excel(df, df2,df21,df3,df31,df4,df5,df6,sheet,writer):
    df.to_excel(writer,sheet_name=sheet,startrow=0 , startcol=0)   
    df2.to_excel(writer,sheet_name=sheet,startrow=len(df.index)+2, startcol=0) 
    df21.to_excel(writer,sheet_name=sheet,startrow=len(df.index)+2, startcol=5) 
    df3.to_excel(writer,sheet_name=sheet,startrow=len(df21.index)+2+len(df.index)+2, startcol=0)
    df31.to_excel(writer,sheet_name=sheet,startrow=len(df21.index)+2+len(df.index)+2, startcol=5)
    df4.to_excel(writer,sheet_name=sheet,startrow=len(df31.index)+2+len(df21.index)+2+len(df.index)+2, startcol=0)
    df5.to_excel(writer,sheet_name=sheet,startrow=len(df4.index)+2+len(df31.index)+2+len(df21.index)+2+len(df.index)+2, startcol=0)
    df6.to_excel(writer,sheet_name=sheet,startrow=0, startcol=11)

In [ ]:
#define main function

def generate_pi_report():
    df=pd.read_csv("input.csv")
    df=df.rename(columns={"Total Media Cost (Advertiser Currency)": "Media Cost"})
    df=df.dropna()
    df=drop_nonshopper(df)
    df=add_name_plate_column(df)
    df=add_strategy_column(df)
    df=add_weight_column(df)
    df=add_pi_column(df)
    df=add_tactic_column(df)
    df=add_browser_family_column(df)
    df=add_os_family_column(df)
    nPlates=df['Name Plate'].unique()
    writer = pd.ExcelWriter('PI report.xlsx', engine='xlsxwriter')
    workbook=writer.book
    money_fmt = workbook.add_format({'num_format': '$#,##0'})
    CPPI_fmt = workbook.add_format({'num_format': '#,##0.00'})
    url_fmt = workbook.add_format({'align': 'right'})
    
    for x in range(nPlates.size):
        worksheet=workbook.add_worksheet(str(nPlates[x]))    
        worksheet.set_zoom(75)
        worksheet.set_column('A:A', 15)
        worksheet.set_column('F:G', 15)
        worksheet.set_column('L2:L10000', 40,url_fmt)
        worksheet.set_column('B:B', 10, money_fmt)
        worksheet.set_column('H:H', 10, money_fmt)
        worksheet.set_column('M:M', 10, money_fmt)
        worksheet.set_column('D:D', 10, CPPI_fmt)
        worksheet.set_column('J:J', 10, CPPI_fmt)
        worksheet.set_column('O:O', 10, CPPI_fmt)
        writer.sheets[str(nPlates[x])]=worksheet
        df1=df[df['Name Plate']==nPlates[x]]
        p1=pivot_device(df1)
        p2=pivot_strategy(df1)
        p21=pivot_strategy_device(df1)
        p3=pivot_tactic(df1)
        p31=pivot_tactic_device(df1)
        p4=pivot_os_family(df1)
        p5=pivot_browser_family(df1)
        p6=pivot_url(df1)
        write_to_excel(p1,p2,p21,p3,p31,p4,p5,p6,str(nPlates[x]),writer) 
        worksheet=workbook.add_worksheet(str(nPlates[x])+" PI Contribution")
        worksheet.set_column('A:A', 40)
        worksheet.set_column('B:B', 20)
        worksheet.set_column('E:E', 20)
        worksheet.set_zoom(75)
        writer.sheets[str(nPlates[x])+" PI Contribution"]=worksheet
        pc1=pivot_pixel(df1)
        if pc1.size>0:
            pc1.to_excel(writer,sheet_name=str(nPlates[x]+" PI Contribution"),startrow=0 , startcol=0) 
    writer.save()
    return df
    

In [ ]:
df=generate_pi_report()

In [ ]:
pivot_tactic(df)

In [ ]:
#plot strategy pivot

strategy_pivot=pivot_strategy(df)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(
        x=strategy_pivot.index,
        y=strategy_pivot['CPPI'],
        name="CPPI",
        line=dict(color='rgba(0, 0, 100, 0.5)', width=1)
    ),secondary_y=True)
fig.add_trace(
    go.Bar(
        x=strategy_pivot.index,
        y=strategy_pivot['Media Cost'],
        name="Media Cost",
        marker=dict(color='lightblue')
    ))